In [2]:
import pandas as pd
import numpy as np
import nfl_data_py as nfl
import datetime as dt
import copy
pd.set_option('display.max_columns', None)


# Background/Ideas

- Features will be based off seasonal,weekly, and career based values
- Idea is that certain players which can be differentiated by career based values paired with weekly performance and or seasonal (team strength proxy) can be paired to build something relatively predictive.
- Interactivity can be dependent on clicking and choosing assortment of players and identifying/projecting current projections.
- Data seems to get updated weekly so these predictions would change over time as well.

# Data

This section focuses on pulling the data and prepping/aggregating the dependent variable. (Fantasy Points)

In [3]:
roster_data = nfl.import_seasonal_rosters([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999])
pbp_df = pd.DataFrame(nfl.import_pbp_data([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))
weekly_df = pd.DataFrame(nfl.import_weekly_data([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))
# injuries_df = pd.DataFrame(nfl.import_injuries([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000]))
schedules_df = pd.DataFrame(nfl.import_schedules([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))

2024 done.
2023 done.
2022 done.
2021 done.
2020 done.
2019 done.
2018 done.
2017 done.
2016 done.
2015 done.
2014 done.
2013 done.
2012 done.
2011 done.
2010 done.
2009 done.
2008 done.
2007 done.
2006 done.
2005 done.
2004 done.
2003 done.
2002 done.
2001 done.
2000 done.
1999 done.
Downcasting floats.
Downcasting floats.


In [125]:
def get_opposing_team(df):
    if df['home_team'] == df['team']:
        val = df['away_team']
    elif df['away_team'] == df['team']:
        val = df['home_team']
    else:
        val = None

    return val


kicker_plays = pbp_df[pbp_df['kicker_player_name'].notnull()]

# kicker_plays['kicking_against_team'] = kicker_plays.apply(get_kicker_opposition,axis = 1)

# Filter out only successful field goals for longest field goal calculation

made_field_goals = kicker_plays[kicker_plays['field_goal_result'] == 'made']


## <40

kicker_plays['sub_40_fg_make'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] == 'made') & (row['kick_distance'] < 40) , 1, 0), axis = 1)
kicker_plays['sub_40_fg_attempt'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] != None) & (row['kick_distance'] < 40) , 1, 0), axis = 1)

## 40-49

kicker_plays['40_49_fg_make'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] == 'made') & (row['kick_distance']  >= 40)  & (row['kick_distance'] < 50), 1, 0), axis = 1)
kicker_plays['40_49_fg_attempt'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result']!= None) & (row['kick_distance']  >= 40)  & (row['kick_distance'] < 50), 1, 0), axis = 1)


## 50+ 

kicker_plays['50_fg_make'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] == 'made') & (row['kick_distance']  >= 50), 1, 0), axis = 1)
kicker_plays['50_fg_attempt'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result']!= None) & (row['kick_distance']  >= 50), 1, 0), axis = 1)





kicker_stats = kicker_plays.groupby(['game_id', 'game_date', 'week', 'season','div_game', 'home_team', 'away_team', 'weather', 'location', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach','kicker_player_id', 'kicker_player_name']).agg({
    'field_goal_result': lambda x: (x == 'made').sum(),  # Sum of successful field goals
    'extra_point_result': lambda x: (x == 'good').sum(),  # Sum of successful extra points
    'kick_distance': 'sum',  # Sum of kickoff yardage
    'field_goal_attempt': 'sum',  # Total field goal attempts
    'kickoff_attempt': 'sum',
    'extra_point_attempt' : 'sum',
    'sub_40_fg_make':'sum',
    'sub_40_fg_attempt':'sum',
    '40_49_fg_make':'sum',
    '40_49_fg_attempt':'sum',
    '50_fg_make':'sum',
    '50_fg_attempt' : 'sum'

}).reset_index()




## Adding in the opponent team for the kicker of interest 

kicker_team = roster_data[['season','player_id','team','depth_chart_position']]

kicker_team['team'] = kicker_team['team'].replace({'OAK':'LV', 'STL':'LA', 'SD':'LAC','HST':'HOU', 'BLT':'BAL', 'CLV':'CLE','SL':'LA','ARZ':'ARI'})

kicker_team.rename(columns = {'player_id':'kicker_player_id'},inplace = True)

kicker_stats = kicker_stats.merge(kicker_team, on = ['kicker_player_id','season'], how = 'inner')


# Calculate longest successful field goal using the filtered made_field_goals DataFrame
longest_fg = made_field_goals.groupby(['game_id','kicker_player_id', 'kicker_player_name'])['kick_distance'].max().reset_index()
longest_fg.rename(columns={'kick_distance': 'longest_fg'}, inplace=True)
# Merge longest field goal into kicker_stats
kicker_stats = pd.merge(kicker_stats, longest_fg, on=['game_id', 'kicker_player_id', 'kicker_player_name'], how='left')

# Rename the aggregated columns for clarity
kicker_stats.rename(columns={
    'field_goal_attempt': 'fg_attempts',
    'field_goal_result': 'fg_made',
    'longest_fg': 'longest_fg',
    'extra_point_result': 'xp_made',
    'extra_point_attempt': 'xp_attempts',
    'kick_distance': 'total_kick_distance'
}, inplace=True)

kicker_stats.drop(columns=['kicker_player_name'], inplace=True)

## Aggregate average score to opposition 

kicker_stats['opponent_team'] = kicker_stats.apply(get_opposing_team,axis = 1)



C:\Users\chris\AppData\Local\Temp\ipykernel_10864\1684507005.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kicker_plays['sub_40_fg_make'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] == 'made') & (row['kick_distance'] < 40) , 1, 0), axis = 1)
C:\Users\chris\AppData\Local\Temp\ipykernel_10864\1684507005.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kicker_plays['sub_40_fg_attempt'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] != None) & (row['kick_d

In [127]:
kicker_stats[kicker_stats['opponent_team'].isna()][['game_id','team','home_team','away_team','opponent_team','season','kicker_player_id']]

,game_id,team,home_team,away_team,opponent_team,season,kicker_player_id
492,2001_12_SD_SEA,BUF,SEA,LAC,None,2001,00-0002975
494,2001_13_SD_PHI,BUF,PHI,LAC,None,2001,00-0002975
496,2001_14_OAK_SD,BUF,LAC,LV,None,2001,00-0002975
498,2001_15_SD_KC,BUF,KC,LAC,None,2001,00-0002975
500,2001_16_SEA_SD,BUF,LAC,SEA,None,2001,00-0002975
...,...,...,...,...,...,...,...
10162,2022_16_CIN_NE,DAL,NE,CIN,None,2022,00-0034909
10454,2023_04_DET_GB,CLE,GB,DET,None,2023,00-0036816
10455,2023_06_DET_TB,CLE,TB,DET,None,2023,00-0036816
10456,2023_07_DET_BAL,CLE,BAL,DET,None,2023,00-0036816


## Basic Kicking Related Variables

In [135]:

# 5 pts per 50+ yard FG made
# 4 pts per 40-49 yard FG made
# 3 pts per FG made, 39 yards or less

def get_opposing_team(df):
    if df['home_team'] == df['team']:
        val = df['away_team']
    elif df['away_team'] == df['team']:
        val = df['home_team']
    else:
        val = None

    return val


kicker_plays = pbp_df[pbp_df['kicker_player_name'].notnull()]

# kicker_plays['kicking_against_team'] = kicker_plays.apply(get_kicker_opposition,axis = 1)

# Filter out only successful field goals for longest field goal calculation

made_field_goals = kicker_plays[kicker_plays['field_goal_result'] == 'made']


## <40

kicker_plays['sub_40_fg_make'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] == 'made') & (row['kick_distance'] < 40) , 1, 0), axis = 1)
kicker_plays['sub_40_fg_attempt'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] != None) & (row['kick_distance'] < 40) , 1, 0), axis = 1)

## 40-49

kicker_plays['40_49_fg_make'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] == 'made') & (row['kick_distance']  >= 40)  & (row['kick_distance'] < 50), 1, 0), axis = 1)
kicker_plays['40_49_fg_attempt'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result']!= None) & (row['kick_distance']  >= 40)  & (row['kick_distance'] < 50), 1, 0), axis = 1)


## 50+ 

kicker_plays['50_fg_make'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] == 'made') & (row['kick_distance']  >= 50), 1, 0), axis = 1)
kicker_plays['50_fg_attempt'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result']!= None) & (row['kick_distance']  >= 50), 1, 0), axis = 1)





kicker_stats = kicker_plays.groupby(['game_id', 'game_date', 'week', 'season','div_game', 'home_team', 'away_team', 'weather', 'location', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach','kicker_player_id', 'kicker_player_name']).agg({
    'field_goal_result': lambda x: (x == 'made').sum(),  # Sum of successful field goals
    'extra_point_result': lambda x: (x == 'good').sum(),  # Sum of successful extra points
    'kick_distance': 'sum',  # Sum of kickoff yardage
    'field_goal_attempt': 'sum',  # Total field goal attempts
    'kickoff_attempt': 'sum',
    'extra_point_attempt' : 'sum',
    'sub_40_fg_make':'sum',
    'sub_40_fg_attempt':'sum',
    '40_49_fg_make':'sum',
    '40_49_fg_attempt':'sum',
    '50_fg_make':'sum',
    '50_fg_attempt' : 'sum'

}).reset_index()




## Adding in the opponent team for the kicker of interest 

kicker_team = roster_data[['season','player_id','team','depth_chart_position']]

kicker_team['team'] = kicker_team['team'].replace({'OAK':'LV', 'STL':'LA', 'SD':'LAC','HST':'HOU', 'BLT':'BAL', 'CLV':'CLE','SL':'LA','ARZ':'ARI'})


kicker_team.rename(columns = {'player_id':'kicker_player_id'},inplace = True)

kicker_stats = kicker_stats.merge(kicker_team, on = ['kicker_player_id','season'], how = 'inner')


# Calculate longest successful field goal using the filtered made_field_goals DataFrame
longest_fg = made_field_goals.groupby(['game_id','kicker_player_id', 'kicker_player_name'])['kick_distance'].max().reset_index()
longest_fg.rename(columns={'kick_distance': 'longest_fg'}, inplace=True)
# Merge longest field goal into kicker_stats
kicker_stats = pd.merge(kicker_stats, longest_fg, on=['game_id', 'kicker_player_id', 'kicker_player_name'], how='left')

# Rename the aggregated columns for clarity
kicker_stats.rename(columns={
    'field_goal_attempt': 'fg_attempts',
    'field_goal_result': 'fg_made',
    'longest_fg': 'longest_fg',
    'extra_point_result': 'xp_made',
    'extra_point_attempt': 'xp_attempts',
    'kick_distance': 'total_kick_distance'
}, inplace=True)

kicker_stats.drop(columns=['kicker_player_name'], inplace=True)

## Aggregate average score to opposition 

kicker_stats['opponent_team'] = kicker_stats.apply(get_opposing_team,axis = 1)

print('Number Missing Opponent:' + str(kicker_stats[kicker_stats['opponent_team'].isna()].shape[0]))
kicker_stats = kicker_stats[~kicker_stats['opponent_team'].isna()]

home_teams = schedules_df[['season','home_team','away_score']].copy()

away_teams = schedules_df[['season','away_team','home_score']].copy()

home_teams.rename(columns = {'home_team':'team','away_score':'points_allowed'}, inplace = True)
away_teams.rename(columns = {'away_team':'team','home_score':'points_allowed'}, inplace = True)

points_allowed_df = pd.concat([home_teams,away_teams])


points_allowed_df['avg_points_allowed'] = points_allowed_df.groupby(['season','team'])['points_allowed'].transform('mean')


points_allowed_df.rename(columns = {'team':'opponent_team'},inplace = True)


kicker_stats = kicker_stats.merge(points_allowed_df[['season','opponent_team','avg_points_allowed']].drop_duplicates(), on = ['opponent_team','season'], how = 'left')

## Creating Average Field Goals Per Game

kicker_stats['average_field_goals'] = kicker_stats.groupby(['kicker_player_id','season'])['fg_made'].transform('mean')

## Creating Field Goal Make Percentage (Seasonal)

kicker_stats['total_made'] = kicker_stats.groupby(['kicker_player_id','season'])['fg_made'].transform('sum')
kicker_stats['total_attempts'] = kicker_stats.groupby(['kicker_player_id','season'])['fg_attempts'].transform('sum')

kicker_stats['season_make_percentage_fg'] = round(kicker_stats['total_made'].astype(float) / kicker_stats['total_attempts'].astype(float),2)


## Creating Average Extra Points Per Game


kicker_stats['average_xp'] = kicker_stats.groupby(['kicker_player_id','season'])['xp_made'].transform('mean')

## Creating Extra Point Make Percentage (Seasonal)

kicker_stats['total_made_xp'] = kicker_stats.groupby(['kicker_player_id','season'])['xp_made'].transform('sum')
kicker_stats['total_attempts_xp'] = kicker_stats.groupby(['kicker_player_id','season'])['xp_attempts'].transform('sum')

kicker_stats['season_make_percentage-xp'] = round(kicker_stats['total_made_xp'].astype(float) / kicker_stats['total_attempts_xp'].astype(float),2)

for_fantasy_scoring = pbp_df[pbp_df['kicker_player_name'].notnull()]


## Game FG Make Percentage

kicker_stats['fg_make_prop'] = kicker_stats['fg_made'] / kicker_stats['fg_attempts']


## 50+ 

kicker_stats['50_make_prop'] = np.where((kicker_stats['50_fg_make'] == 0 ) & (kicker_stats['50_fg_attempt'] == 0), 0 ,kicker_stats['50_fg_make'] / kicker_stats['50_fg_attempt'])


## 40-49

kicker_stats['40_49_make_prop'] = np.where((kicker_stats['40_49_fg_make'] == 0 ) & (kicker_stats['40_49_fg_attempt'] == 0),0,kicker_stats['40_49_fg_make'] / kicker_stats['40_49_fg_attempt'])

## Sub 40 

kicker_stats['sub_40_make_prop'] = np.where((kicker_stats['sub_40_fg_make'] == 0 ) & (kicker_stats['sub_40_fg_attempt'] == 0),0,kicker_stats['sub_40_fg_make'] / kicker_stats['sub_40_fg_attempt'])



## Creating Average Extra Point Attempts (Proxy for how often Team Scores)


kicker_stats['xp_attempts_per_game'] = round(kicker_stats.groupby(['kicker_player_id','season'])['xp_attempts'].transform('mean'),2)


## Average Fantasy Points Allowed To Kickers


## scoring criteria 
# https://www.espn.com/fantasy/football/ffl/story?page=fflrulesstandardscoring

kicker_stats['total_kicking_points_fantasy'] = (kicker_stats['50_fg_make'] * 5) + (kicker_stats['40_49_fg_make'] * 4) + (kicker_stats['sub_40_fg_make'] * 3) +((kicker_stats['40_49_fg_attempt'] - kicker_stats['40_49_fg_make']) * -1) + ((kicker_stats['sub_40_fg_attempt'] - kicker_stats['sub_40_fg_make']) * -2) + (kicker_stats['xp_made'] * 1)

# .apply(lambda row: (int(row['50_fg_make']) * 5) + (row['40_49_fg_make'] * 4) + (row['sub_40_fg_make'] * 3) + ((row['fg_attempts'] - row['fg_made']) * -1)) 


kicker_stats['opp_avg_kick_points_allowed'] = kicker_stats.groupby('opponent_team')['total_kicking_points_fantasy'].transform('mean')
 

# kicker_stats['points_from_kicks'] = kicker_stats['fg_made']
# kicker_stats['points_from_xp'] = kick


C:\Users\chris\AppData\Local\Temp\ipykernel_10864\2644166056.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kicker_plays['sub_40_fg_make'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] == 'made') & (row['kick_distance'] < 40) , 1, 0), axis = 1)
C:\Users\chris\AppData\Local\Temp\ipykernel_10864\2644166056.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kicker_plays['sub_40_fg_attempt'] = kicker_plays.apply(lambda row: np.where((row['field_goal_result'] != None) & (row['kick_d

Number Missing Opponent:352


## Weather/Condition Related Variables

- Home/Away Encoding : Chirps and Energy From Home Games could increase Pressure
- Surface (One Hot Encoding): The Surface of the field is something to monitor.
- Interaction term with MPH of Wind and Make Percentage (Windier Conditions can have interaction with lower fg make percentage)

In [140]:
kicker_stats['home_away_flag'] = np.where(kicker_stats['opponent_team'] == kicker_stats['home_team'], 0,1)
kicker_stats['surface'] = kicker_stats['surface'].str.strip()
kicker_stats = kicker_stats.join(pd.get_dummies(data = kicker_stats['surface'], prefix = 'flag'))

In [141]:
## Wind MPH INteraction and Game Kicking Percentage 

kicker_stats['wind_make_interaction'] = kicker_stats['wind'] * kicker_stats['fg_make_prop']
kicker_stats['wind_make_interaction'] = kicker_stats['wind_make_interaction'].fillna(0)

## Aggregating Proportion of Scoring From Kicks

- Rationale is that if a team is showing signs of scoring mainly from Touchdowns and 2 Point Conversions kickers are likely to have less influence.
- Also if this is aggregated at the game level, interaction with wind and or other weather related factors could prove to be useful.

In [143]:
game_w_score_df = schedules_df[['away_score','home_score','home_team','away_team','game_id']].copy()
## Cleaveland and Baltimore Bug In Denotions across Schedules DF and the Rest


# game_w_score_df['home_team'] = game_w_score_df['home_team'].replace({'CLE':'CLV', 'BAL':'BLT'})
# game_w_score_df['away_team'] = game_w_score_df['away_team'].replace({'CLE':'CLV', 'BAL':'BLT'})


kicker_stats = kicker_stats.merge(game_w_score_df, how = 'inner', on = ['game_id', 'home_team','away_team'])



kicker_stats['team_actual_scored_points'] = np.where(kicker_stats['team'] == kicker_stats['home_team'], kicker_stats['home_score'],kicker_stats['away_score'])

## Filtering out bad data from incorrect schedule game result values

kicker_stats = kicker_stats[~(kicker_stats['team_actual_scored_points'] == 0) & ((kicker_stats['fg_made'] > 0) | (kicker_stats['xp_made'] > 0))]
                            

kicker_stats.drop(columns = ['home_score','away_score'],inplace = True)

## Game Level
kicker_stats['points_scored_from_kicks_prop'] = ((kicker_stats['fg_made'] * 3) + (kicker_stats['xp_made'] * 1 )) /  kicker_stats['team_actual_scored_points'] 

## Season Team Level

kicker_stats['points_scored_from_kicks_szn'] = kicker_stats.groupby(['team','season'])['points_scored_from_kicks_prop'].transform('mean') 

## Number of Games Played in Career

kicker_stats['games_played_in_career'] = kicker_stats.groupby('kicker_player_id')['game_id'].transform('count')




# Begin Modeling

In [200]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_absolute_error,mean_squared_error
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
 

In [161]:
x_vars = ['div_game','spread_line','avg_points_allowed','average_field_goals','total_made','total_attempts',
          'average_xp', '50_make_prop','40_49_make_prop','sub_40_make_prop','xp_attempts_per_game',
          'opp_avg_kick_points_allowed','home_away_flag','flag_a_turf','flag_astroplay','flag_astroturf','flag_dessograss','flag_fieldturf',
          'flag_grass','flag_sportturf','wind_make_interaction','team_actual_scored_points','points_scored_from_kicks_prop','points_scored_from_kicks_szn',
          'games_played_in_career'  ]

y_var = ['total_kicking_points_fantasy']

In [147]:
## Checking For Nulls

[x for x in kicker_stats.columns[kicker_stats.isna().any()].tolist() if x in x_vars]

[]

## First Approach Basic Stepwise For Feature Selection

In [177]:
len(x_vars)

25

In [187]:

clf = linear_model.LinearRegression()

# Build step forward feature selection
sfs1 = sfs(clf,k_features = 10,forward=True,floating=False, scoring='neg_mean_absolute_error', cv = 5)

# Perform SFFS
sfs1 = sfs1.fit(kicker_stats[x_vars], kicker_stats[y_var])

# selected_features = sfs.fit(, )

In [189]:
results = pd.DataFrame.from_dict(sfs1.get_metric_dict()).T
[x for x in results[results['avg_score'] == results['avg_score'].max()]['feature_names']]


# results

[('average_field_goals',
  'total_attempts',
  'average_xp',
  '50_make_prop',
  '40_49_make_prop',
  'sub_40_make_prop',
  'xp_attempts_per_game',
  'team_actual_scored_points',
  'points_scored_from_kicks_prop',
  'points_scored_from_kicks_szn')]

## Training Linear Regression With Selected Features

In [207]:
from sklearn.model_selection import train_test_split
from math import sqrt
# import root_mean_squared_error, r2_score,mean_absolute_error

selected_features = ['average_field_goals',
  'total_attempts',
  'average_xp',
  '50_make_prop',
  '40_49_make_prop',
  'sub_40_make_prop',
  'xp_attempts_per_game',
  'team_actual_scored_points',
  'points_scored_from_kicks_prop',
  'points_scored_from_kicks_szn']


y_var = ['total_kicking_points_fantasy']

X_train, X_test, y_train, y_test = train_test_split(kicker_stats[selected_features], kicker_stats[y_var], test_size=0.2, random_state=42)

In [203]:
model = linear_model.LinearRegression()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)

In [211]:
## MSE

mse = mean_squared_error(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)


rmse = sqrt(mse)

print("MSE: " + str(mse) + ' \nMAE: ' + str(mae) + ' \nRMSE: ' + str(rmse))

MSE: 3.074463780902373 
MAE: 1.3284207148849263 
RMSE: 1.7534148912628673


## EBM With L1 Regularization for Feature Selection

- Blackbox model with more predictive power 
- L1 (Lasso) for feature selection